# Using Pandas and Metadb
The popular [Pandas](https://pandas.pydata.org/) data analysis and manipulation tool can be used with the FOLIO [MetaDB](https://librarydataplatform.org/) within a Jupyter notebook. 

## Install additional dependencies
1. Pandas `pip install pandas`
1. Install [SQLalchemy](https://www.sqlalchemy.org/) `pip install psycopg2-binary`

In [3]:
import os
metadb_url = os.getenv("METADB_URL")
user = os.getenv("METADB_USER")
password = os.getenv("METADB_PASSWORD")

In [5]:
import pandas as pd

from sqlalchemy import create_engine

In [6]:
metadb_engine = create_engine("postgresql+psycopg2://{0}:{1}@{2}/{3}".format(
    user,
    password,
    metadb_url,
    "okapi"))

In [7]:
metadb_connection = metadb_engine.connect()

## Search for Contributor Type

In [9]:
contributor_types_sql = """SELECT id, code, name FROM folio_inventory.contributor_type__t__;"""
contributor_types_df = pd.read_sql(contributor_types_sql,
                                   con=metadb_connection)

In [10]:
contributor_types_df

,id,code,name
0,0efdaf72-6126-430a-8256-69c42ff6866f,tcd,Technical director
1,6901fbf1-c038-42eb-a03e-cd65bf91f660,dgg,Degree granting institution
2,21dda3dc-cebd-4018-8db2-4f6d50ce3d02,own,Owner
3,81c01802-f61b-4548-954a-22aab027f6e5,clr,Colorist
4,5d92d9de-adf3-4dea-93b5-580e9a88e696,cpc,Copyright claimant
...,...,...,...
263,81bbe282-dca7-4763-bf5a-fe28c8939988,pro,Producer
264,2576c328-61f1-4684-83cf-4376a66f7731,fld,Field director
265,c6005151-7005-4ee7-8d6d-a6b72d25377a,vdg,Videographer
266,a986c8f2-b36a-400d-b09f-9250a753563c,brl,Braille embosser


## Instances with it's Holdings Call Numbers

In [ ]:
instance_holdings_sql = """SELECT instance__t__.id, instance__t__.title, holdings_record__t__.call_number, folio_inventory.location__t__.code
FROM folio_inventory.instance__t__, folio_inventory.holdings_record__t__, folio_inventory.location__t__
WHERE folio_inventory.holdings_record__t__.instance_id = folio_inventory.instance__t__.id
AND folio_inventory.holdings_record__t__.permanent_location_id = folio_inventory.location__t__.id
LIMIT 250;"""